In [ ]:
def getExteriorCategory(extr):
    if extr == 'AsbShng':
        return 0
    elif extr == 'AsphShn':
        return 1
    elif extr == 'BrkComm':
        return 2
    elif extr == 'BrkFace':
        return 3
    elif extr == 'CBlock':
        return 4
    elif extr == 'CemntBd':
        return 5
    elif extr == 'HdBoard':
        return 6
    elif extr == 'ImStucc':
        return 7
    elif extr == 'MetalSd':
        return 8
    elif extr == 'Other':
        return 9
    elif extr == 'Plywood':
        return 10
    elif extr == 'PreCast':
        return 11
    elif extr == 'Stone':
        return 12
    elif extr == 'Stucco':
        return 13 
    elif extr == 'VinylSd':
        return 14
    elif extr == 'Wd Sdng':
        return 15
    elif extr == 'WdShing':
        return 16   

'''
       Blmngtn  Bloomington Heights
       Blueste  Bluestem
       BrDale   Briardale
       BrkSide  Brookside
       ClearCr  Clear Creek
       CollgCr  College Creek
       Crawfor  Crawford
       Edwards  Edwards
       Gilbert  Gilbert
       IDOTRR   Iowa DOT and Rail Road
       MeadowV  Meadow Village
       Mitchel  Mitchell
       NAmes    North Ames
       NoRidge  Northridge
       NPkVill  Northpark Villa
       NridgHt  Northridge Heights
       NWAmes   Northwest Ames
       OldTown  Old Town
       SWISU    South & West of Iowa State University
       Sawyer   Sawyer
       SawyerW  Sawyer West
       Somerst  Somerset
       StoneBr  Stone Brook
       Timber   Timberland
       Veenker  Veenker
'''
def getSortedNeighborCategory(nbr):
    if nbr == '':
        return 0
    elif nbr == 'MeadowV':
        return 1    
    elif nbr == 'IDOTRR':
        return 2    
    elif nbr == 'BrDale':
        return 3    
    elif nbr == 'OldTown':
        return 4   
    elif nbr == 'Edwards':
        return 5    
    elif nbr == 'BrkSide':
        return 6 
    elif nbr == 'Sawyer':
        return 7 
    elif nbr == 'Blueste':
        return 8     
    elif nbr == 'SWISU':
        return 9
    elif nbr == 'NAmes':
        return 10  
    elif nbr == 'NPkVill':
        return 11   
    elif nbr == 'Mitchel':
        return 12 
    elif nbr == 'SawyerW':
        return 13    
    elif nbr == 'Gilbert':
        return 14   
    elif nbr == 'NWAmes':
        return 15    
    elif nbr == 'Blmngtn':
        return 16    
    elif nbr == 'CollgCr':
        return 17  
    elif nbr == 'ClearCr':
        return 18    
    elif nbr == 'Crawfor':
        return 19    
    elif nbr == 'Veenker':
        return 20
    elif nbr == 'Somerst':
        return 21    
    elif nbr == 'Timber':
        return 22  
    elif nbr == 'StoneBr':
        return 23    
    elif nbr == 'NoRidge':
        return 24    
    elif nbr == 'NridgHt':
        return 25    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# comma delimited is the default
hsData = pd.read_csv(train_file, header = 0)

hsData['Overall_QQ'] = hsData['OverallQual']*10 + hsData['OverallCond']

hsData['House_Freshment'] = hsData['YearBuilt'] + hsData['YearRemodAdd']
hsData['House_Freshment'].apply(lambda year: year - 3700)

hsData['FireplaceQu'] = hsData['FireplaceQu'].fillna('NA')
hsData['FireplaceQu'].replace(['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], [0,1,2,3,4,5], inplace=True)

hsData['Neighborhood_sort_ID'] = hsData['Neighborhood'].apply(getSortedNeighborCategory)

hsData['MSZoning_ID'] = hsData['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [0,1,2,3,4])
hsData['MSSubClass_ID'] = hsData['MSSubClass'].replace([180,30,45,190,50,90,85,40,160,70,20,75,80,120,60], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
hsData['MSZMSS_ID'] = hsData['MSSubClass_ID']*10 + hsData['MSZoning_ID']

hsData['Total_FlrSF'] = hsData['1stFlrSF'] + hsData['2ndFlrSF']
#hsData = hsData[hsData["Total_FlrSF"] < 4000]
#hsData = hsData[hsData["Total_FlrSF"] > 500]
#hsData = hsData[hsData["GrLivArea"] < 2500]
#hsData = hsData[hsData["Overall_QQ"] < 100]
#hsData = hsData[hsData["Overall_QQ"] > 30]

hsData['PorchSF'] = hsData['OpenPorchSF'] + hsData['EnclosedPorch'] + hsData['3SsnPorch'] + hsData['ScreenPorch']

hsData['Foundation'].replace(['Wood', 'Slab', 'Stone', 'CBlock', 'BrkTil', 'PConc'], [0,1,2,3,4,5], inplace=True)
hsData['LotShape'].replace(['Reg', 'IR1', 'IR2', 'IR3'], [0,1,2,3], inplace=True)
hsData['KitchenQual'].replace(['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], [0,1,2,3,4,5], inplace=True)

hsData['Exter_QQ'] = hsData['ExterQual'] + hsData['ExterCond']
'''
print hsData['Exter_QQ'].unique()
hsData['Exter_QQ'].replace(['FaFa', 'TAPo', 'FaTA', 'TAFa', 'TAEx', 'TATA', 'TAGd', 'GdGd', \
                             'GdTA', 'ExGd', 'ExEx', 'ExTA'], \
                             [61,77,99,119,140,140,140,210,221,267,325,374], inplace=True)
'''

hsData['GarageType'].replace(['NA', 'CarPort', 'Detchd', 'Basment', '2Types', 'Attchd', 'BuiltIn'], [0,1,2,3,4,5,6], inplace=True)

hsData['CentralAir'].replace(['N', 'Y'], [0,1], inplace=True)
hsData['HeatingQC'].replace(['Po', 'Fa', 'TA', 'Gd', 'Ex'], [1,2,3,4,5], inplace=True)
hsData['Air_OQ'] = hsData['CentralAir']*10 + hsData['HeatingQC']

# Conditions are not helpful
'''
hsData['Conditions'] = hsData['Condition1'] + hsData['Condition2']
hsData['Conditions'].replace(['FeedrFeedr', 'FeedrRRNn', 'ArteryArtery', 'ArteryNorm', \
                              'RRNnFeedr', 'RRAnFeedr', 'FeedrRRAn', 'FeedrNorm', \
                             'RRAeNorm', 'NormNorm', 'RRAnNorm', 'FeedrRRAe', \
                             'RRNeNorm', 'PosNNorm', 'PosANorm', 'RRNnNorm', \
                             'PosNPosN', 'ArteryPosA'], \
                             [85,97,107,120,128,134,137,141,141,167,179,190,190,200,212,252,285,325], inplace=True)
'''
hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['Bsmt_QQ'] = hsData['BsmtQual'] + hsData['BsmtCond']

'''
print hsData['Bsmt_QQ'].unique()
hsData['Bsmt_QQ'].replace(['FaPo', 'NANA', 'FaTA', 'FaFa', 'TAFa', 'TATA', 'TAGd', 'GdFa', \
                            'GdTA', 'GdGd', 'ExGd', 'ExTA'], \
                             [64,112,114,114,114,135,164,182,192,199,275,318], inplace=True)
'''
#0.27 before
hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['Garage_QQ'] = hsData['GarageQual'] + hsData['GarageCond']
'''
hsData['Garage_QQ'].replace(['PoPo', 'NANA', 'FaFa', 'FaPo', 'FaTA', 'ExEx', 'TAFa', 'TAGd', \
                             'TATA', 'GdTA', 'GdGd', 'ExTA'], \
                             [97,100,110,116,116,125,125,144,171,209,227,475], inplace=True)
'''

hsData['HouseStyle'].replace(['1.5Unf', '1.5Fin', '2.5Unf', 'SFoyer', '1Story', 'SLvl', '2Story', '2.5Fin'], [0,1,1,1,2,3,4,5], inplace=True)
hsData['Total_Bath'] = hsData['FullBath'] + hsData['BsmtFullBath'] + hsData['HalfBath']*0.5 + hsData['BsmtHalfBath']*0.5
#hsData.info()
#hsData.describe()


In [ ]:
numeric_features = hsData.select_dtypes(include=[np.number])
numeric_features.dtypes

corr = numeric_features.corr()

print (corr['SalePrice'].sort_values(ascending=False)[:30], '\n')
print (corr['SalePrice'].sort_values(ascending=False)[-30:])

In [ ]:
#hsData_oqq_trim = hsData
hsData_oqq_trim = hsData[hsData["Overall_QQ"] < 100]
hsData_oqq_trim = hsData_oqq_trim[hsData_oqq_trim["Overall_QQ"] > 30]

plt.scatter(x=hsData_oqq_trim["Overall_QQ"], y=np.log(hsData_oqq_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Overall QQ")
plt.show()

In [ ]:
st_bar = hsData.pivot_table(index="SaleType", values="SalePrice", aggfunc=np.median)

st_bar_sorted = st_bar.sort_values(by="SalePrice")
st_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("SaleType")
plt.ylabel("sale price")
plt.show()

In [ ]:
hs_bar = hsData.pivot_table(index="HouseStyle", values="SalePrice", aggfunc=np.median)

hs_bar_sorted = hs_bar.sort_values(by="SalePrice")
hs_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("HouseStyle")
plt.ylabel("sale price")
plt.show()



In [ ]:
nb_bar = hsData.pivot_table(index="Neighborhood", values="SalePrice", aggfunc=np.median)

nb_bar_sorted = nb_bar.sort_values(by="SalePrice")
nb_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("neighborhood")
plt.ylabel("sale price")
plt.show()

In [ ]:
msz_bar = hsData.pivot_table(index="MSZoning", values="SalePrice", aggfunc=np.median)

msz_bar_sorted = msz_bar.sort_values(by="SalePrice")
msz_bar_sorted.plot(kind="bar", color="red")

plt.xlabel("ms zoning")
plt.ylabel("sale price")
plt.show()

In [ ]:
mss_bar = hsData.pivot_table(index="MSSubClass", values="SalePrice", aggfunc=np.median)

mss_bar_sorted = mss_bar.sort_values(by="SalePrice")
mss_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("ms subclass")
plt.ylabel("sale price")
plt.show()


In [ ]:
tMeadowV = hsData.loc[hsData['Neighborhood'] == 'MeadowV','SalePrice']
tIDOTRR = hsData.loc[hsData['Neighborhood'] == 'IDOTRR','SalePrice']
tBrDale = hsData.loc[hsData['Neighborhood'] == 'BrDale','SalePrice']
tOldTown = hsData.loc[hsData['Neighborhood'] == 'OldTown','SalePrice']
tEdwards = hsData.loc[hsData['Neighborhood'] == 'Edwards','SalePrice']
tBrkSide = hsData.loc[hsData['Neighborhood'] == 'BrkSide','SalePrice']
tSawyer = hsData.loc[hsData['Neighborhood'] == 'Sawyer','SalePrice']
tBlueste = hsData.loc[hsData['Neighborhood'] == 'Blueste','SalePrice']
tSWISU = hsData.loc[hsData['Neighborhood'] == 'SWISU','SalePrice']
tNAmes = hsData.loc[hsData['Neighborhood'] == 'NAmes','SalePrice']
tNPkVill = hsData.loc[hsData['Neighborhood'] == 'NPkVill','SalePrice']
tMitchel = hsData.loc[hsData['Neighborhood'] == 'Mitchel','SalePrice']
tSawyerW = hsData.loc[hsData['Neighborhood'] == 'SawyerW','SalePrice']
tGilbert = hsData.loc[hsData['Neighborhood'] == 'Gilbert','SalePrice']
tNWAmes = hsData.loc[hsData['Neighborhood'] == 'NWAmes','SalePrice']
tBlmngtn = hsData.loc[hsData['Neighborhood'] == 'Blmngtn','SalePrice']
tCollgCr = hsData.loc[hsData['Neighborhood'] == 'CollgCr','SalePrice']
tClearCr = hsData.loc[hsData['Neighborhood'] == 'ClearCr','SalePrice']
tCrawfor = hsData.loc[hsData['Neighborhood'] == 'Crawfor','SalePrice']
tVeenker = hsData.loc[hsData['Neighborhood'] == 'Veenker','SalePrice']
tSomerst = hsData.loc[hsData['Neighborhood'] == 'Somerst','SalePrice']
tTimber = hsData.loc[hsData['Neighborhood'] == 'Timber','SalePrice']
tStoneBr = hsData.loc[hsData['Neighborhood'] == 'StoneBr','SalePrice']
tNoRidge = hsData.loc[hsData['Neighborhood'] == 'NoRidge','SalePrice']
tNridgHt = hsData.loc[hsData['Neighborhood'] == 'NridgHt','SalePrice']

#1
print("tMeadowV median:%s, mean:%s, std:%s" % (tMeadowV.median(), tMeadowV.mean(), tMeadowV.std()))
#2
print("tIDOTRR median:%s, mean:%s, std:%s" % (tIDOTRR.median(), tIDOTRR.mean(), tIDOTRR.std()))
print("tBrDale median:%s, mean:%s, std:%s" % (tBrDale.median(), tBrDale.mean(), tBrDale.std()))
#3
print("tOldTown median:%s, mean:%s, std:%s" % (tOldTown.median(), tOldTown.mean(), tOldTown.std()))
print("tEdwards median:%s, mean:%s, std:%s" % (tEdwards.median(), tEdwards.mean(), tEdwards.std()))
print("tBrkSide median:%s, mean:%s, std:%s" % (tBrkSide.median(), tBrkSide.mean(), tBrkSide.std()))
#4
print("tSawyer median:%s, mean:%s, std:%s" % (tSawyer.median(), tSawyer.mean(), tSawyer.std()))
print("tBlueste median:%s, mean:%s, std:%s" % (tBlueste.median(), tBlueste.mean(), tBlueste.std()))
print("tSWISU median:%s, mean:%s, std:%s" % (tSWISU.median(), tSWISU.mean(), tSWISU.std()))
print("tNAmes median:%s, mean:%s, std:%s" % (tNAmes.median(), tNAmes.mean(), tNAmes.std()))
#5
print("tNPkVill median:%s, mean:%s, std:%s" % (tNPkVill.median(), tNPkVill.mean(), tNPkVill.std()))
#6
print("tMitchel median:%s, mean:%s, std:%s" % (tMitchel.median(), tMitchel.mean(), tMitchel.std()))
#7
print("tSawyerW median:%s, mean:%s, std:%s" % (tSawyerW.median(), tSawyerW.mean(), tSawyerW.std()))
print("tGilbert median:%s, mean:%s, std:%s" % (tGilbert.median(), tGilbert.mean(), tGilbert.std()))
print("tNWAmes median:%s, mean:%s, std:%s" % (tNWAmes.median(), tNWAmes.mean(), tNWAmes.std()))
#8
print("tBlmngtn median:%s, mean:%s, std:%s" % (tBlmngtn.median(), tBlmngtn.mean(), tBlmngtn.std()))
#9
print("tCollgCr median:%s, mean:%s, std:%s" % (tCollgCr.median(), tCollgCr.mean(), tCollgCr.std()))
print("tClearCr median:%s, mean:%s, std:%s" % (tClearCr.median(), tClearCr.mean(), tClearCr.std()))
print("tCrawfor median:%s, mean:%s, std:%s" % (tCrawfor.median(), tCrawfor.mean(), tCrawfor.std()))
#10
print("tVeenker median:%s, mean:%s, std:%s" % (tVeenker.median(), tVeenker.mean(), tVeenker.std()))
#11
print("tSomerst median:%s, mean:%s, std:%s" % (tSomerst.median(), tSomerst.mean(), tSomerst.std()))
print("tTimber median:%s, mean:%s, std:%s" % (tTimber.median(), tTimber.mean(), tTimber.std()))
#12
print("tStoneBr median:%s, mean:%s, std:%s" % (tStoneBr.median(), tStoneBr.mean(), tStoneBr.std()))
#13
print("tNoRidge median:%s, mean:%s, std:%s" % (tNoRidge.median(), tNoRidge.mean(), tNoRidge.std()))
#14
print("tNridgHt median:%s, mean:%s, std:%s" % (tNridgHt.median(), tNridgHt.mean(), tNridgHt.std()))

In [ ]:
cd1_bar = hsData.pivot_table(index="Condition1", values="SalePrice", aggfunc=np.median)

cd1_bar_sorted = cd1_bar.sort_values(by="SalePrice")
cd1_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("condition 1")
plt.ylabel("sale price")
plt.show()

'''
119.55
140.0
142.5
166.5
171.495
190.75
200.0
212.5
214.0
'''
print np.median(hsData.loc[hsData['Condition1'] == 'Artery','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'Feedr','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'RRAe','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'Norm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'RRAn','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'RRNe','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'PosN','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'PosA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition1'] == 'RRNn','SalePrice'])/1000

#hsData['Condition1'].replace(['Artery', 'Feedr', 'RRAe', 'Norm', 'RRAn', 'RRNe', 'PosN', 'PosA', 'RRNn'], [120,140,143,166,171,191,200,212,214], inplace=True)

In [ ]:
cd2_bar = hsData.pivot_table(index="Condition2", values="SalePrice", aggfunc=np.median)

cd2_bar_sorted = cd2_bar.sort_values(by="SalePrice")
cd2_bar_sorted.plot(kind="bar", color="yellow")

plt.xlabel("condition 2")
plt.ylabel("sale price")
plt.show()

'''
96.75
106.5
127.5
136.905
163.5
190.0
284.875
325.0
'''
print np.median(hsData.loc[hsData['Condition2'] == 'RRNn','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'Artery','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'Feedr','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'RRAn','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'Norm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'RRAe','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'PosN','SalePrice'])/1000
print np.median(hsData.loc[hsData['Condition2'] == 'PosA','SalePrice'])/1000

#hsData['Condition1'].replace(['RRNn', Artery', 'Feedr', 'RRAn', 'Norm', 'RRAe', 'PosN', 'PosA'], [97,107,128,137,164,190,284,325], inplace=True)

In [ ]:
cds_bar = hsData.pivot_table(index="Conditions", values="SalePrice", aggfunc=np.median)

cds_bar_sorted = cds_bar.sort_values(by="SalePrice")
cds_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("condition 1+2")
plt.ylabel("sale price")
plt.show()

'''
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
'''
print np.median(hsData.loc[hsData['Conditions'] == 'FeedrFeedr','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'FeedrRRNn','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'ArteryArtery','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'ArteryNorm','SalePrice'])/1000

print np.median(hsData.loc[hsData['Conditions'] == 'RRNnFeedr','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'RRAnFeedr','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'FeedrRRAn','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'FeedrNorm','SalePrice'])/1000

print np.median(hsData.loc[hsData['Conditions'] == 'RRAeNorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'NormNorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'RRAnNorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'FeedrRRAe','SalePrice'])/1000

print np.median(hsData.loc[hsData['Conditions'] == 'RRNeNorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'PosNNorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'PosANorm','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'RRNnNorm','SalePrice'])/1000

print np.median(hsData.loc[hsData['Conditions'] == 'PosNPosN','SalePrice'])/1000
print np.median(hsData.loc[hsData['Conditions'] == 'ArteryPosA','SalePrice'])/1000

'''
hsData['Conditions'].replace(['FeedrFeedr', 'FeedrRRNn', 'ArteryArtery', 'ArteryNorm', \
                              'RRNnFeedr', 'RRAnFeedr', 'FeedrRRAn', 'FeedrNorm', \
                             'RRAeNorm', 'NormNorm', 'RRAnNorm', 'FeedrRRAe', \
                             'RRNeNorm', 'PosNNorm', 'PosANorm', 'RRNnNorm', \
                             'PosNPosN', 'ArteryPosA'], \
                             [85,97,107,120,128,134,137,141,141,167,179,190,190,200,212,252,285,325], inplace=True)
'''

In [ ]:
bsm_bar = hsData.pivot_table(index="Bsmt_QQ", values="SalePrice", aggfunc=np.median)

bsm_bar_sorted = bsm_bar.sort_values(by="SalePrice")
bsm_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Bsmt Qual+Cond")
plt.ylabel("sale price")
plt.show()

print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'FaPo','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'NANA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'FaTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'FaFa','SalePrice'])/1000

print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'TAFa','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'TATA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'TAGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'GdFa','SalePrice'])/1000

print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'GdTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'GdGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'ExGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Bsmt_QQ'] == 'ExTA','SalePrice'])/1000

'''
hsData['Bsmt_QQ'].replace(['FaPo', 'NANA', 'FaTA', 'FaFa', 'TAFa', 'TATA', 'TAGd', 'GdFa', \
                            'GdTA', 'GdGd', 'ExGd', 'ExTA'], \
                             [64,112, 114,114,114,135,164,182,192,199,275,318], inplace=True)
'''

In [ ]:
gg_bar = hsData.pivot_table(index="Garage_QQ", values="SalePrice", aggfunc=np.median)

gg_bar_sorted = gg_bar.sort_values(by="SalePrice")
gg_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Garage Qual+Cond")
plt.ylabel("sale price")
plt.show()

print np.median(hsData.loc[hsData['Garage_QQ'] == 'PoPo','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'NANA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'FaFa','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'FaPo','SalePrice'])/1000

print np.median(hsData.loc[hsData['Garage_QQ'] == 'FaTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'ExEx','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'TAFa','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'TAGd','SalePrice'])/1000

print np.median(hsData.loc[hsData['Garage_QQ'] == 'TATA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'GdTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'GdGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Garage_QQ'] == 'ExTA','SalePrice'])/1000

'''
hsData['Garage_QQ'].replace(['PoPo', 'NANA', 'FaFa', 'FaPo', 'FaTA', 'ExEx', 'TAFa', 'TAGd', \
                             'TATA', 'GdTA', 'GdGd', 'ExTA'], \
                             [97,100,110,116,116,125,125,144,171,209,227,475], inplace=True)
'''

In [ ]:
ext_bar = hsData.pivot_table(index="Exter_QQ", values="SalePrice", aggfunc=np.median)

ext_bar_sorted = ext_bar.sort_values(by="SalePrice")
ext_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Exterior Qual+Cond")
plt.ylabel("sale price")
plt.show()

print np.median(hsData.loc[hsData['Exter_QQ'] == 'FaFa','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'TAPo','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'FaTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'TAFa','SalePrice'])/1000

print np.median(hsData.loc[hsData['Exter_QQ'] == 'TAEx','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'TATA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'TAGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'GdGd','SalePrice'])/1000

print np.median(hsData.loc[hsData['Exter_QQ'] == 'GdTA','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'ExGd','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'ExEx','SalePrice'])/1000
print np.median(hsData.loc[hsData['Exter_QQ'] == 'ExTA','SalePrice'])/1000

'''
hsData['Exter_QQ'].replace(['FaFa', 'TAPo', 'FaTA', 'TAFa', 'TAEx', 'TATA', 'TAGd', 'GdGd', \
                             'GdTA', 'ExGd', 'ExEx', 'ExTA'], \
                             [61,77,99,119,140,140,140,210,221,267,325,374], inplace=True)
'''

In [ ]:
gt_bar = hsData.pivot_table(index="GarageType", values="SalePrice", aggfunc=np.median)

gt_bar_sorted = gt_bar.sort_values(by="SalePrice")
gt_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Garage location")
plt.ylabel("sale price")
plt.show()

In [ ]:
# Living area
plt.scatter(x=hsData["GrLivArea"], y=np.log(hsData["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Above grade (ground) living area square feet")
plt.show()
# there are 2 strange values, their are are so big but the price is low

In [ ]:
# Trimmed Living area
hsData_liv_trim = hsData[hsData["GrLivArea"] < 2000]

plt.scatter(x=hsData_liv_trim["GrLivArea"], y=np.log(hsData_liv_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Sale Price")
plt.xlabel("Above grade (ground) living area square feet")
plt.show()

# there are 2 strange values, their are are so big but the price is low

In [ ]:
# Garage area

plt.scatter(x=hsData["GarageArea"], y=np.log(hsData["SalePrice"]))
plt.grid(True)
plt.ylabel("Sale Price")
plt.xlabel("Size of garage in square feet")
plt.show()

In [ ]:
#hsData_gar_trim = hsData[hsData["GarageArea"] < 1200]

plt.scatter(x=hsData["GarageArea"], y=np.log(hsData["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Size of garage in square feet")
plt.show()

#hsData_trim.info()

In [ ]:
plt.scatter(x=hsData['YearBuilt'], y=np.log(hsData['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

In [ ]:
hsData_year_shift = hsData['House_Freshment']

plt.scatter(x=hsData_year_shift, y=np.log(hsData['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()


In [ ]:
plt.scatter(x=hsData["Total_FlrSF"], y=np.log(hsData["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Floor square feet")
plt.show()

In [ ]:
hsData_flr_trim = hsData[hsData["Total_FlrSF"] < 3000]
hsData_flr_trim = hsData_flr_trim[hsData_flr_trim["Total_FlrSF"] > 500]

plt.scatter(x=hsData_flr_trim["Total_FlrSF"], y=np.log(hsData_flr_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Floor square feet")
plt.show()